# entropy_ops

> Ops for computing two-body entropies.

In [ ]:
#| default_exp entropy_ops

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

import torch


def pointwise_shannon(ps, eps=1e-20):
    log_ps = torch.log2(ps + eps)

    return -ps * log_ps


def smooth_mean_one_body_entropy(x: torch.Tensor) -> torch.Tensor:
    """Smooth extension of the plug-in estimator of the onr-body Shannon entropy.
    `x` must have shape (..., N, L, R). The result has shape (...,)."""
    assert x.ndim >= 3
    n_samples = x.shape[-3]

    one_body_counts = x.sum(dim=-3)
    one_body_freqs = one_body_counts / n_samples
    # For each position, compute the Shannon entropy of the corresponding one-body
    # distribution. Then, average over all positions.
    mean_one_body_entr = pointwise_shannon(one_body_freqs).sum(-1).mean(-1)

    return mean_one_body_entr


def smooth_mean_two_body_entropy(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """Smooth extension of the plug-in estimator of the two-body Shannon entropy.
    `x` must have shape (..., N, L, R), and `y` must have shape (N, L, R).
    The result has shape (...,)."""
    assert x.ndim >= 3 and y.ndim == 3
    assert x.shape[-3] == y.shape[-3]
    n_samples = x.shape[-3]

    two_body_counts = torch.einsum("...nia,njb->...iajb", x, y)
    two_body_freqs = two_body_counts / n_samples
    # For each pair of positions, compute the Shannon entropy of the corresponding two-body
    # distribution. Then, average over all pairs of positions.
    mean_two_body_entr = pointwise_shannon(two_body_freqs).sum((-3, -1)).mean((-2, -1))

    return mean_two_body_entr

In [ ]:
show_doc(smooth_mean_two_body_entropy)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/entropy_ops.py#L31){target="_blank" style="float:right; font-size:smaller"}

### smooth_mean_two_body_entropy

>      smooth_mean_two_body_entropy (x:torch.Tensor, y:torch.Tensor)

Smooth extension of the plug-in estimator of the two-body Shannon entropy.
`x` must have shape (..., N, L, R), and `y` must have shape (N, L, R).
The result has shape (...,).